# 📊 Predicción de Churn en Telecom

Este notebook desarrolla el flujo de trabajo del proyecto final de **Metodología para Data Science**.

Dataset: [Telco Customer Churn (IBM)](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)


In [ ]:
# Instalación de librerías (si es necesario)
# !pip install pandas numpy scikit-learn xgboost matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [ ]:
# Cargar dataset (ajusta la ruta a donde guardes el archivo)
df = pd.read_csv('data/Telco-Customer-Churn.csv')

print("Dimensiones:", df.shape)
df.head()

In [ ]:
# Preprocesamiento básico

# Eliminar filas con valores vacíos en TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])

# Codificación de variables categóricas
df_encoded = pd.get_dummies(df.drop('customerID', axis=1), drop_first=True)

# Separar variables predictoras y objetivo
X = df_encoded.drop('Churn_Yes', axis=1)
y = df_encoded['Churn_Yes']

# Escalado de variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print("Conjunto de entrenamiento:", X_train.shape, "Conjunto de prueba:", X_test.shape)

In [ ]:
# Entrenamiento de modelos

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)


In [ ]:
# Función de evaluación
def evaluar_modelo(y_test, y_pred, nombre):
    print(f"\nModelo: {nombre}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, y_pred))
    print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    plt.title(f"Matriz de Confusión - {nombre}")
    plt.show()

# Evaluar los tres modelos
evaluar_modelo(y_test, y_pred_lr, "Logistic Regression")
evaluar_modelo(y_test, y_pred_rf, "Random Forest")
evaluar_modelo(y_test, y_pred_xgb, "XGBoost")

## ✅ Conclusiones iniciales

- Se entrenaron tres modelos: Logistic Regression, Random Forest y XGBoost.
- Las métricas clave fueron **Accuracy**, **Recall** (para churn) y **ROC-AUC**.
- El siguiente paso será comparar los resultados y definir el modelo con mejor desempeño.
